In [1]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# # stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [2]:

acc=[]
for i in open('/home/trishalaswain/ClientData/Other/Accounts.txt','r').readlines():
    acc.append(i.rstrip('\n'))

# fac=[16,253,21,22,28,37,40,38,39,245,201,389,380,219,220,221,205,203,251,258
# ]



In [3]:
# acc=['660003246401'
# ]

In [4]:
len(acc)

1576

In [6]:
# type_id=[1563054,1563000,1564044,1564165,1563384,1564042,1564157,1564299]

In [7]:
# type_id

In [8]:
df_all=[]

# for f_id in type_id:
query ='''
SELECT DISTINCT
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    dpd.id AS DPD_ID,
    dm.path AS filePath,
    dm.id as document_id,
    em.id as encounter_id
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON dm.account_id = em.account_id
        AND dm.service_date = em.service_date
        AND dm.physician_id = em.physician_id
        AND em.is_active = 1
WHERE
    is_physician_document = 1
        AND dm.is_active = 1
        and account_number in ({})
        and dpd.id = (select MAX(id)FROM document_processing_detail
          WHERE document_id = dm.id)

ORDER BY dm.created_date DESC;
'''.format(','.join(['"'+str(x)+'"' for x in acc]))
# print(query)

# for f in fac:

# query='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         left JOIN
#     document_code dc ON dc.document_id = dm.id
#         left JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dc.code_category = 'EANDM'
#         and is_physician_document=1
#         and dm.is_active=1
#         and account_number in ({})
#         and dpd.id = (SELECT MAX(id)
#           FROM document_processing_detail
#           WHERE document_id = dm.id)
#     order by dm.created_date desc;
# '''.format(','.join(str(x) for x in acc))
           
df_all.append(pd.read_sql(query, con=db_connection))
print(len(df_all))


1


In [9]:
df_query=pd.concat(df_all)

In [10]:
df_query

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_request_s3_path,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,DPD_ID,filePath,document_id,encounter_id
0,V00001046054,ezcapc/PH/data/CHINO/nlp/cdp-xml/20230512/8782...,ezcapc/PH/data/CHINO/nlp/ner/20230512/8782688_...,221,2023-05-12 07:00:00,ezcapc/PH/data/CHINO/nlp/pcs-xml/20230512/8782...,ezcapc/PH/data/CHINO/preprocessing/20230512/87...,ezcapc/PH/data/CHINO/preprocessing/20230512/87...,PH/data/CHINO/document/20230512/CVMC_V00001046...,64115,...,ezcapc/PH/data/CHINO/eandmcs/20230512/8319402_...,ezcapc/PH/data/CHINO/eandmcs/20230512/8319402_...,ezcapc/PH/data/CHINO/ruleengine/20230512/53903...,ezcapc/PH/data/CHINO/cptcs/20230512/8712135_CV...,ezcapc/PH/data/CHINO/cptcs/20230512/8712135_CV...,8585326,8870483,PH/data/CHINO/document/20230512/CVMC_V00001046...,42104145,7017687
1,V00001044665,ezcapc/PH/data/CHINO/nlp/cdp-xml/20230502/8781...,ezcapc/PH/data/CHINO/nlp/ner/20230502/8781588_...,221,2023-05-02 07:00:00,ezcapc/PH/data/CHINO/nlp/pcs-xml/20230502/8781...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,PH/data/CHINO/document/20230502/CVMC_V00001044...,64115,...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318406_...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318406_...,ezcapc/PH/data/CHINO/ruleengine/20230502/53897...,ezcapc/PH/data/CHINO/cptcs/20230502/8711101_CV...,ezcapc/PH/data/CHINO/cptcs/20230502/8711101_CV...,8308392,8869377,PH/data/CHINO/document/20230502/CVMC_V00001044...,42099920,7016778
2,V00001044683,ezcapc/PH/data/CHINO/nlp/cdp-xml/20230502/8781...,ezcapc/PH/data/CHINO/nlp/ner/20230502/8781580_...,221,2023-05-02 07:00:00,ezcapc/PH/data/CHINO/nlp/pcs-xml/20230502/8781...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,PH/data/CHINO/document/20230502/CVMC_V00001044...,64115,...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318398_...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318398_...,ezcapc/PH/data/CHINO/ruleengine/20230502/53897...,ezcapc/PH/data/CHINO/cptcs/20230502/8711091_CV...,ezcapc/PH/data/CHINO/cptcs/20230502/8711091_CV...,8308402,8869370,PH/data/CHINO/document/20230502/CVMC_V00001044...,42099914,7016774
3,V00001044691,ezcapc/PH/data/CHINO/nlp/cdp-xml/20230502/8781...,ezcapc/PH/data/CHINO/nlp/ner/20230502/8781575_...,221,2023-05-02 07:00:00,ezcapc/PH/data/CHINO/nlp/pcs-xml/20230502/8781...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,PH/data/CHINO/document/20230502/CVMC_V00001044...,64115,...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318377_...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318377_...,ezcapc/PH/data/CHINO/ruleengine/20230502/53896...,ezcapc/PH/data/CHINO/cptcs/20230502/8711064_CV...,ezcapc/PH/data/CHINO/cptcs/20230502/8711064_CV...,8308408,8869338,PH/data/CHINO/document/20230502/CVMC_V00001044...,42099890,7016762
4,V00001044662,ezcapc/PH/data/CHINO/nlp/cdp-xml/20230502/8781...,ezcapc/PH/data/CHINO/nlp/ner/20230502/8781571_...,221,2023-05-02 07:00:00,ezcapc/PH/data/CHINO/nlp/pcs-xml/20230502/8781...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,ezcapc/PH/data/CHINO/preprocessing/20230502/87...,PH/data/CHINO/document/20230502/CVMC_V00001044...,64115,...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318372_...,ezcapc/PH/data/CHINO/eandmcs/20230502/8318372_...,ezcapc/PH/data/CHINO/ruleengine/20230502/53896...,ezcapc/PH/data/CHINO/cptcs/20230502/8711058_CV...,ezcapc/PH/data/CHINO/cptcs/20230502/8711058_CV...,8308389,8869335,PH/data/CHINO/document/20230502/CVMC_V00001044...,42099887,7016759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694,V00001042063,ezcapc/PH/data/CHINO/nlp/cdp-xml/20230414/7963...,ezcapc/PH/data/CHINO/nlp/ner/20230414/7963029_...,221,2023-04

In [11]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [12]:
df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id']).first().reset_index()

In [13]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [14]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id', 'DPD_ID',
       'document_id', 'encounter_id'],
      dtype='object')

In [15]:
print(df_query.groupby('facility_id').size().to_markdown())

|   facility_id |    0 |
|--------------:|-----:|
|           221 | 1699 |


In [16]:
# commands

In [24]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id, dpd_id,t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             # df_query.suggested_code, df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_pcs.xml".format(env, pcs, fac_name,enc_id, acc, t_id))
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_eandm_request.json".format(env, em_req,fac_name, enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name,dpd_id, acc, t_id)))
        cmd.append((em_req, '{}/{}/{}_{}_eandm_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, dpd_id,t_id))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd```````_id, enc_id, acc)))
        cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.pdf".format(env,ori_path,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc))
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))
        cmd.append((nlp_req, '{}/{}/{}_{}_nlp_req.json'.format(fac_name,dpd_id, acc, t_id)))
        cmd.append((nlp_res, '{}/{}/{}_{}_nlp_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [25]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("//home/trishalaswain/Documents/analysis/data/download_scripts_and_tars/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [26]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [27]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [28]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

[221]
aws s3 cp 's3://ezdi-production-bucket/ezcapc/PH/data/CHINO/nlp/pcs-xml/20230308/7442589_CVMC_V00001036860_EMERGENCY DEPT REPORT_03_28_2023_22.22.38.PDF_20230329120717.017_result.xml' CHINO/6013078/V00001036860_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/PH/data/CHINO/eandmcs/20230308/7116474_CVMC_V00001036860_EMERGENCY DEPT REPORT_03_28_2023_22.22.38.PDF_20230329120746.046_request.json' CHINO/6013078/V00001036860_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/PH/data/CHINO/cptcs/20230308/7427105_CVMC_V00001036860_EMERGENCY DEPT REPORT_03_28_2023_22.22.38.PDF_20230329120733.033_request.json' CHINO/6013078/7539344_64115_cpt_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/PH/data/CHINO/nlp/cdp-xml/20230308/7442589_CVMC_V00001036860_EMERGENCY DEPT REPORT_03_28_2023_22.22.38.PDF_20230329120707.007_request.json' CHINO/6013078/V00001036860_NLPrequest.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/PH/data/CHINO/nlp/ner/20230308/7442589_CVMC_V00

""


In [29]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id', 'DPD_ID',
       'document_id', 'encounter_id'],
      dtype='object')

In [30]:
# df_query[df_query.account_number.str.contains('0996')]

In [ ]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()